In [ ]:
# import modules
import os, sys
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from tqdm import tqdm

In [ ]:
# load data
directory = '/home/user/data'

path = os.chdir(directory)

image_D = os.path.join(directory, 'D.nii') # load Diffusion map
image_pseudoD = os.path.join(directory, 'pseudoD.nii') # load Pseudo Diffusion map
image_f = os.path.join(directory, 'f.nii') # load Perfusion Fraction map

Nvoxels = dim1*dim2*dim3 # dimensions

D = nib.load(image_D).get_fdata().reshape((Nvoxels,))
pseudoD = nib.load(image_pseudoD).get_fdata().reshape((Nvoxels,))
f = nib.load(image_f).get_fdata().reshape((Nvoxels,))

D_nz = []
pseudoD_nz = []
f_nz = []

for i in range(Nvoxels):
    if D[i] != 0. or pseudoD[i] != 0. or f[i] != 0.:
        D_nz.append(D[i])
        pseudoD_nz.append(pseudoD[i])
        f_nz.append(f[i])

In [ ]:
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

class OutliersDetection:
        
    # algorithms of detection
    def elliptic_envelope(data):
        cov = EllipticEnvelope(random_state=0).fit(data)
        out = []
        arr = cov.predict(data) 
        for i in range(len(data[:,0])):
            if arr[i] == -1:
                out.append([data[i,0], data[i,1], data[i,2]])
        return out
                        
    def isolation_forest(data):
        cov = IsolationForest(random_state=0).fit(data)
        out = []
        arr = cov.predict(data) 
        for i in range(len(data[:,0])):
            if arr[i] == -1:
                out.append([data[i,0], data[i,1], data[i,2]])
        return out
    
    def local_outlier_factor(data):
        clf = LocalOutlierFactor(n_neighbors=2)
        out = []
        arr = clf.fit_predict(data) 
        for i in range(len(data[:,0])):
            if arr[i] == -1:
                out.append([data[i,0], data[i,1], data[i,2]])
        return out

In [ ]:
data = np.array([D_nz,pseudoD_nz,f_nz]).transpose()
outliers = OutliersDetection.elliptic_envelope(data) # choose the algorithm